In [1]:
#Computation Graphs and Automatic Differentiation
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [20]:
# tracing your result, and how it came to be :: requires_grad=True

# Tensor factory methods have a ``requires_grad`` flag
a = torch.tensor([1., 2., 3], requires_grad=True)
k = 10
# With requires_grad=True, you can still do all the operations you previously
# could
b = torch.tensor([4., 5., 6], requires_grad=True)
c = k*a + b
print(c)
print(c.grad_fn,  '\n')

# So Tensors know what created them. c knows that it wasn’t read in from a file, 
# it was the result of a multiplication or exponential or whatever. 
# And if you keep following c.grad_fn, you will find yourself at x and y.

#going a bit deeper
s = c.sum()
print(s)
print(s.grad_fn)
s.backward()
print('a grad',a.grad) # ds / da
print('b grad',b.grad) # ds / db


x = torch.tensor([[1., 2., 3.]], requires_grad=True)
print(x.size())
y = torch.tensor([[4.], [5.], [6.]], requires_grad=True)
print(y.size())

z = x .mm(y)
print(z)
# BUT z knows something extra.
print(z.grad_fn)

z1 = y .mm(x)
print(z1)
print(z1.grad_fn)



tensor([14., 25., 36.], grad_fn=<AddBackward0>)

tensor(75., grad_fn=<SumBackward0>)
a grad tensor([10., 10., 10.])
b grad tensor([1., 1., 1.])
torch.Size([1, 3])
torch.Size([3, 1])
tensor([[32.]], grad_fn=<MmBackward>)
tensor([[ 4.,  8., 12.],
        [ 5., 10., 15.],
        [ 6., 12., 18.]], grad_fn=<MmBackward>)


In [21]:
x = torch.randn(2, 2)
y = torch.randn(2, 2)
# By default, user created Tensors have ``requires_grad=False``
print(x.requires_grad, y.requires_grad)
z = x + y
# So you can't backprop through z
print('requires grad, not set ::',z.grad_fn)

# ``.requires_grad_( ... )`` changes an existing Tensor's ``requires_grad``
# flag in-place. The input flag defaults to ``True`` if not given.
x = x.requires_grad_()
y = y.requires_grad_()
# z contains enough information to compute gradients, as we saw above
z = x + y
# If any input to an operation has ``requires_grad=True``, so will the output
print('requires grad transferring with operation ::',z.requires_grad, z.grad_fn)


new_z = z
print('requires grad transferring with assignment ::', new_z.grad_fn)


# Now z has the computation history that relates itself to x and y
# Can we just take its values, and **detach** it from its history?
new_z = z.detach()

# ... does new_z have information to backprop to x and y?
# NO!
print('requires grad not set after detachment ::', new_z.grad_fn)
# And how could it? ``z.detach()`` returns a tensor that shares the same storage
# as ``z``, but with the computation history forgotten. It doesn't know anything
# about how it was computed.
# In essence, we have broken the Tensor away from its past history

False False
requires grad, not set :: None
requires grad transferring with operation :: True <AddBackward0 object at 0x000001188F2B8C48>
requires grad transferring with assignment :: <AddBackward0 object at 0x000001188F2B8C48>
requires grad not set after detachment :: None
